In [ ]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

In [ ]:
dataset["validation"][0]

# Collect CONLL statistics
- number of samples
- number of tokens per sample
- number of entities per sample
- ratio of O tags to non-O tags

In [ ]:
len(dataset["train"])

In [ ]:
len(dataset["validation"])

In [ ]:
len(dataset["test"])

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
data = dataset["train"].map(lambda sample: {"length":len(sample["tokens"])})["length"]

In [ ]:
kde = gaussian_kde(data)

In [ ]:
bins = plt.hist(data, bins=50, density=True)
xx = np.linspace(0, bins[1][-1], 100)
plt.plot(xx, kde(xx))
plt.title("Distribution of Number of Tokens per Sample (Train set)")

In [ ]:
# Mode
bins[1][np.argmax(bins[0])], bins[1][np.argmax(bins[0])+1]

In [ ]:
np.mean(data)

In [ ]:
np.percentile(data, [5, 25, 50, 75, 95])

In [ ]:
data = dataset["train"].map(lambda sample: {"n_chars":len(' '.join(sample["tokens"]))})["n_chars"]
char_bins = plt.hist(data, bins=100, density=True)
kde = gaussian_kde(data)
xx = np.linspace(0, char_bins[1][-1], 100)
plt.plot(xx, kde(xx))

In [ ]:
char_bins

In [ ]:
short = dataset["train"].filter(lambda datum: len(" ".join(datum["tokens"]))<20)


In [ ]:
len(short)

In [ ]:
import random

In [ ]:
idx = random.randint(0, len(short))
short[idx]

In [ ]:
dataset

In [ ]:
dataset["train"][50:55]["tokens"]

In [ ]:
pd.to_pickle(kde, "char_kde.pkl")

In [ ]:
# Mode
char_bins[1][np.argmax(char_bins[0])], char_bins[1][np.argmax(char_bins[0])+1]

In [ ]:
n_chars = np.array(dataset["train"].map(lambda sample: {"n_chars":len(' '.join(sample["tokens"]))})["n_chars"])

In [ ]:
np.mean(n_chars)

In [ ]:
np.percentile(n_chars,[5, 25, 50, 75, 95])

In [ ]:
n_chars

## num entities

In [ ]:
CONLL_TAG_MAP = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
CONLL_TAG_MAP

In [ ]:
per_ents = dataset["train"].map(lambda sample: {"num_PER":sum([label == 1 for label in sample["ner_tags"]])})["num_PER"]

In [ ]:
sum(per_ents)

In [ ]:
bins = plt.hist(per_ents, bins=np.arange(0, 20, 1), density=True);
kde = gaussian_kde(per_ents)
xx = np.arange(1, bins[1][-1],1)
plt.plot(xx, kde(xx))
plt.title("Number of PER entities per sample")

In [ ]:
org_ents = dataset["train"].map(lambda sample: {"num_ORG":sum([label == 3 for label in sample["ner_tags"]])})["num_ORG"]
sum(org_ents)

In [ ]:
bins = plt.hist(org_ents, bins=np.arange(0, 10, 1), density=True);
kde = gaussian_kde(org_ents)
xx = np.arange(1, bins[1][-1],1)
plt.plot(xx, kde(xx))
plt.title("Number of ORG entities per sample")

In [ ]:
loc_ents = dataset["train"].map(lambda sample: {"num_LOC":sum([label == 5 for label in sample["ner_tags"]])})["num_LOC"]
sum(loc_ents)

In [ ]:
bins = plt.hist(loc_ents, bins=np.arange(0, 10, 1), density=True);
kde = gaussian_kde(loc_ents)
xx = np.arange(1, bins[1][-1],1)
plt.plot(xx, kde(xx))
plt.title("Number of LOC entities per sample")

In [ ]:
ratio_O = dataset["train"].map(lambda sample: {"ratio_O":np.mean([label == 0 for label in sample["ner_tags"]])})["ratio_O"]

In [ ]:
num_O = dataset["train"].map(lambda sample: {"num_O":np.sum([label == 0 for label in sample["ner_tags"]])})["num_O"]
num_total = dataset["train"].map(lambda sample: {"num_O":len(sample["ner_tags"])})["num_O"]

In [ ]:
sum(num_O) / sum(num_total)

In [ ]:
bins = plt.hist(ratio_O, bins=np.linspace(0, 1, 20), density=True);

kde = gaussian_kde(ratio_O)
xx = np.linspace(0, 1, 50)
plt.plot(xx, kde(xx))
plt.title("Ratio of 'O' Labels per Sample")

In [ ]:
num_no_ents = dataset["train"].map(lambda sample: {"no_ents":np.all([label == 0 for label in sample["ner_tags"]])})["no_ents"]
sum(num_no_ents)

In [ ]:
sum(num_no_ents) / len(dataset["train"])

Takeaways
- Apparently, very decent NER models have only a few thousand samples for each Entity category
- samples are typically quite short: 95% have fewer than 37 tokens, 50% have fewer than 10 tokens
- most samples dont cantain an entity from a given class
- when a sample contains an entity, it usually only contains one of a given class
- on average, training samples have 0.83% "O" tags
- 20% of samples have NO ENTITIES!!